In [1]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np

from argos import translate

/tmp/ipykernel_1230519/1412947759.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Split into parts

In [3]:
data = pd.read_csv('../data/en-fr.csv', dtype={'en': 'string', 'pl': 'string'}, na_values=[""])
data['fr'] = ''
data = data.rename(columns={'fr': 'pl'})
data = data.dropna(how='any', subset=['en'])

In [14]:
def split_dataframe(df, n):
    rows_per_dataframe = len(df) // n
    indices = np.arange(0, len(df), rows_per_dataframe)
    split_dataframes = [df.iloc[i:i+rows_per_dataframe] for i in indices]
    return split_dataframes

In [15]:
parts = split_dataframe(data, 10)
for idx, part in enumerate(parts):
    part.to_csv(f'../data/en-pl-{idx}.csv', index=False)

## Translate single part

In [22]:
def translate_part(part_path, save_interval=1000):
    data = pd.read_csv(part_path, dtype={'en': 'string', 'pl': 'string'}, na_values=[""])
    for i, row in tqdm(data.iterrows(), total=len(data)):
        if not pd.isna(row['pl']):
            # skip if translation is already done
            continue
        try:
            data.at[i, 'pl'] = translate(row['en'])
        except Exception:
            print('Error while translating', repr(row['en']))
        
        if i % save_interval == 0:
            data.to_csv(part_path, index=False)

In [ ]:
translate_part('../data/en-pl-2.csv')

## Join parts

In [6]:
def join(parts_dir, output_path):
    parts_paths = list(sorted(Path(parts_dir).glob('en-pl-*.csv'), key=lambda p: p.name))
    parts = []
    for path in tqdm(parts_paths):
        part = pd.read_csv(path, dtype={'en': 'string', 'pl': 'string'}, na_values=[""])
        part = part[~part['pl'].isna()]
        parts.append(part)
    joined = pd.concat(parts)
    joined.to_csv(output_path, index=False)
    return joined

In [7]:
joined = join('../data', '../data/en-pl.csv')

100%|██████████| 10/10 [00:41<00:00,  4.15s/it]


In [9]:
joined.head()

,en,pl
0,Changing Lives | Changing Society | How It Wor...,Zmiana życia 124; Zmiana społeczeństwa 124; Ja...
1,Site map,Mapa strony
2,Feedback,Informacje zwrotne
3,Credits,Kredyty
4,Français,Français
